<a href="https://colab.research.google.com/github/harishiker99/SmolLM-Classification/blob/main/SmolLM_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate torch datasets tqdm
!pip install bitsandbytes peft trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import time
import re
from tqdm import tqdm


In [ ]:
from datasets import load_dataset, Dataset
import random

# Load dataset from Hugging Face
dataset = load_dataset("harishnair04/SemEval2024-Task8-SubtaskA")

# Use the "train" split (it contains labels), and convert to a list to allow shuffling
train_data = dataset["train"]
eval_data = dataset["test"]

In [ ]:
train_data

Dataset({
    features: ['text', 'label', 'model', 'source', 'id'],
    num_rows: 119757
})

In [ ]:
eval_data

Dataset({
    features: ['text', 'label', 'model', 'source', 'id'],
    num_rows: 34272
})

In [ ]:

train_list = train_data.to_list()

# Shuffle the list randomly
random.shuffle(train_list)

# Convert back to a Hugging Face Dataset
shuffled_dataset = Dataset.from_list(train_list)

# Split into train/test (e.g., 80% train, 20% test)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2)

# You now have:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]


# Shuffle the dataset randomly
shuffled_train_dataset = train_dataset.shuffle(seed=42)  # seed for reproducibility
shuffled_test_dataset = test_dataset.shuffle(seed=42)

# Select a smaller random subset
small_train_dataset = shuffled_train_dataset.select(range(30000))
small_test_dataset = shuffled_test_dataset.select(range(5000))

# You now have:
train_dataset = small_train_dataset
test_dataset = small_test_dataset

In [ ]:
from transformers import AutoTokenizer

checkpoint = "HuggingFaceTB/SmolLM-1.7B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token


# Tokenize function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

# Apply tokenization to both train and test
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Remove the original "text" column, and format for PyTorch
train_dataset = train_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])
train_dataset.set_format("torch")
test_dataset.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

# Load tokenizer and add pad token if needed
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# Load 8-bit quantized model
bnb_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)

# Set pad_token_id
model.config.pad_token_id = tokenizer.pad_token_id


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM-1.7B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import prepare_model_for_kbit_training

# Enable gradient checkpointing & prepare for LoRA
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# Define LoRA config
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # works for LLaMA-style models
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"
)

# Inject LoRA adapters
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Shows % of parameters actually being trained


trainable params: 3,149,824 || all params: 1,714,530,304 || trainable%: 0.1837


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_dir="./logs",
    report_to="none",  # or "wandb" if using W&B
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)



No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
500,0.737600
1000,0.452800
1500,0.478600
2000,0.423300
2500,0.395600
3000,0.332000
3500,0.396500
4000,0.282500
4500,0.242300
5000,0.276800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reen

TrainOutput(global_step=15000, training_loss=0.29068866729736326, metrics={'train_runtime': 7026.7652, 'train_samples_per_second': 4.269, 'train_steps_per_second': 2.135, 'total_flos': 1.859174793216e+16, 'train_loss': 0.29068866729736326, 'epoch': 1.0})

In [ ]:
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


{'eval_loss': 0.21104951202869415,
 'eval_runtime': 324.4954,
 'eval_samples_per_second': 15.409,
 'eval_steps_per_second': 7.704,
 'epoch': 1.0}

In [ ]:
model.save_pretrained("smollm-lora-human-vs-machine")
tokenizer.save_pretrained("smollm-lora-human-vs-machine")


('smollm-lora-human-vs-machine/tokenizer_config.json',
 'smollm-lora-human-vs-machine/special_tokens_map.json',
 'smollm-lora-human-vs-machine/vocab.json',
 'smollm-lora-human-vs-machine/merges.txt',
 'smollm-lora-human-vs-machine/added_tokens.json',
 'smollm-lora-human-vs-machine/tokenizer.json')

Inference

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load base quantized model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "HuggingFaceTB/SmolLM-1.7B",
    quantization_config=bnb_config,
    device_map="auto"
)

# Attach LoRA adapters
model = PeftModel.from_pretrained(base_model, "smollm-lora-human-vs-machine")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("smollm-lora-human-vs-machine")
tokenizer.pad_token = tokenizer.eos_token

# ✅ Important fix
model.config.pad_token_id = tokenizer.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM-1.7B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
eval_data

Dataset({
    features: ['text', 'label', 'model', 'source', 'id'],
    num_rows: 34272
})

In [ ]:
print(eval_data[0])

{'text': "Today, many adults or teenage drivers are hooked onto their phones. While driving, they can be prompted to use their phones for text messaging. It may cause many accidents, death, serious injuries and more. I honestly think that drivers should use phones while driving  because they are taking risks that could kill or injure others and also yourself. There are also laws against using a phone while operating a moving vehicle but people still disobey them. I think that there should be more consequences when it comes down to texting and driving.  Using your cell phones causes many distractions. It only takes a blink of an eye to cause an accident. Yeah, resisting the urge to text while driving may be hard but it can and will save lives including yours. When driving, your eyes and mind are programmed to be focused on the road at all times. Having a cellphone on your person is a hazard in my opinion. Just think about it, anyone could be crossing a busy street. The person walking is

In [ ]:
from collections import Counter

sources = [row["source"] for row in dataset["test"]]
print(Counter(sources))


Counter({None: 34272})


In [ ]:

# Prepare evaluation dataset
# Drop label/model/source ONLY if they exist
eval_data = eval_data.remove_columns([col for col in eval_data.column_names if col not in ["id", "text"]])




In [ ]:
eval_data

Dataset({
    features: ['text', 'id'],
    num_rows: 34272
})

In [ ]:
# Just test the tokenizer output on one sample
sample = eval_data[0]
print(tokenizer(sample["text"], padding="max_length", truncation=True, max_length=64))


{'input_ids': [7886, 28, 800, 3598, 355, 21591, 9346, 359, 31941, 4581, 480, 10631, 30, 2445, 5828, 28, 502, 416, 325, 17147, 288, 722, 480, 10631, 327, 1694, 21187, 30, 657, 654, 1728, 800, 10736, 28, 2112, 28, 3199, 6345, 284, 540, 30, 339, 22816, 1510, 338, 9346, 868, 722, 10631, 979, 5828, 216, 975, 502, 359, 2498, 4143, 338, 856, 5619, 355, 39894, 1449, 284], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
def preprocess(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=64)

eval_data = eval_data.map(preprocess, batched=True)
eval_data.set_format(type="torch", columns=["input_ids", "attention_mask", "id"])


Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

predictions = []
dataloader = DataLoader(eval_data, batch_size=8)

with torch.no_grad():
    for i, batch in enumerate(tqdm(dataloader)):
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().tolist()

        batch_ids = eval_data[i * 8 : i * 8 + len(preds)]["id"]
        for id_val, label in zip(batch_ids, preds):
                predictions.append({
                 "id": id_val if isinstance(id_val, str) else str(id_val),
                "label": int(label)  # 👈 convert tensor to int
                   })

100%|██████████| 4284/4284 [11:17<00:00,  6.32it/s]


In [ ]:
with open("Results.jsonl", "w") as f:
    for entry in predictions:
        f.write(json.dumps(entry) + "\n")

print("✅ Results.jsonl created!")


✅ Results.jsonl created!


In [ ]:
import pandas as pd
import torch
import json

# 🔄 Load the existing prediction file
df = pd.read_json("Results.jsonl", lines=True)

# 🔧 Convert tensor IDs to int or str
def clean_id(val):
    if isinstance(val, torch.Tensor):
        return int(val.item())
    elif hasattr(val, "item"):  # e.g. numpy
        return int(val.item())
    elif isinstance(val, str) and val.startswith("tensor("):
        return int(val.replace("tensor(", "").replace(")", ""))
    return int(val)

df["id"] = df["id"].apply(clean_id)

# ✅ Save back to Results.jsonl
with open("Results.jsonl", "w") as f:
    for row in df.to_dict(orient="records"):
        f.write(json.dumps({"id": str(row["id"]), "label": int(row["label"])}) + "\n")

print("✅ Fixed Results.jsonl saved with clean IDs!")


✅ Fixed Results.jsonl saved with clean IDs!


Results Saved

Evaluation


In [ ]:
from datasets import load_dataset

# Load official test set (provided JSONL)
eval_data2 = load_dataset("json", data_files="subtaskA_monolingual.jsonl")#, split="train")


In [ ]:
from datasets import load_dataset
import json

# Load the full dataset (no split needed)
dataset_eval = load_dataset("json", data_files="subtaskA_monolingual.jsonl", split="train")

# Save as JSONL (e.g., gold file for evaluation)
with open("subtaskA_gold_labels.jsonl", "w") as f:
    for row in dataset_eval:
        f.write(json.dumps({
            "id": str(row["id"]),
            "label": int(row["label"])
        }) + "\n")

print("✅ Saved as subtaskA_gold_labels.jsonl")


✅ Saved as subtaskA_gold_labels.jsonl


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score

# === Load Predictions ===
pred_df = pd.read_json("Results.jsonl", lines=True)

# === Load Gold Labels (official dev/test set) ===
gold_df = pd.read_json("subtaskA_monolingual.jsonl", lines=True)

# === Keep only 'id' and 'label' from gold set
gold_df = gold_df[["id", "label"]]

In [ ]:
print(pred_df)

          id  label
0          0      0
1          1      1
2          2      1
3          3      1
4          4      1
...      ...    ...
34267  34267      0
34268  34268      1
34269  34269      1
34270  34270      0
34271  34271      1

[34272 rows x 2 columns]


In [ ]:
gold_df

,id,label
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
34267,34267,0
34268,34268,1
34269,34269,1
34270,34270,0


In [ ]:


# === Ensure IDs are strings and trimmed
pred_df["id"] = pred_df["id"].astype(str).str.strip()
gold_df["id"] = gold_df["id"].astype(str).str.strip()

# === 🔍 DEBUG: Show samples, types, and lengths
print("\n🔍 Sample Prediction IDs:", pred_df["id"].head(5).tolist())
print("🔍 Sample Gold IDs:", gold_df["id"].head(5).tolist())

print("\n🔢 ID Types:")
print("Predictions:", pred_df["id"].dtype)
print("Gold      :", gold_df["id"].dtype)

print("\n📏 ID Lengths (Top 5):")
print("Pred:", pred_df['id'].apply(len).value_counts().head())
print("Gold :", gold_df['id'].apply(len).value_counts().head())

# === 🔁 Check for intersection
common_ids = set(pred_df["id"]) & set(gold_df["id"])
print(f"\n🔁 Number of matching IDs: {len(common_ids)}")
if len(common_ids) > 0:
    print("✅ Sample matches:", list(common_ids)[:5])
else:
    print("❌ No matching IDs found. Check your prediction source!")

# === Merge and Evaluate
merged = pred_df.merge(gold_df, on="id", suffixes=("_pred", "_gold"))
print(f"\n✅ Merged {len(merged)} rows out of {len(gold_df)} gold labels.")

if merged.empty:
    raise ValueError("❌ No matching IDs found between prediction and gold file!")

# === 🎯 Compute Metrics
macro_f1 = f1_score(merged["label_gold"], merged["label_pred"], average="macro", zero_division=0)
micro_f1 = f1_score(merged["label_gold"], merged["label_pred"], average="micro", zero_division=0)
accuracy = accuracy_score(merged["label_gold"], merged["label_pred"])

# === 📊 Show Results
print("\n📈 Evaluation Metrics:")
print(f"Accuracy  : {accuracy:.5f}")
print(f"Macro-F1  : {macro_f1:.5f}")
print(f"Micro-F1  : {micro_f1:.5f}")



🔍 Sample Prediction IDs: ['0', '1', '2', '3', '4']
🔍 Sample Gold IDs: ['0', '1', '2', '3', '4']

🔢 ID Types:
Predictions: object
Gold      : object

📏 ID Lengths (Top 5):
Pred: id
5    24272
4     9000
3      900
2       90
1       10
Name: count, dtype: int64
Gold : id
5    24272
4     9000
3      900
2       90
1       10
Name: count, dtype: int64

🔁 Number of matching IDs: 34272
✅ Sample matches: ['28002', '14002', '21275', '13873', '4762']

✅ Merged 34272 rows out of 34272 gold labels.

📈 Evaluation Metrics:
Accuracy  : 0.81772
Macro-F1  : 0.81403
Micro-F1  : 0.81772


In [ ]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    classification_report,
)

# Load fixed prediction file
pred_df = pd.read_json("Results.jsonl", lines=True)
gold_df = pd.read_json("subtaskA_monolingual.jsonl", lines=True)[["id", "label"]]

# Clean IDs
pred_df["id"] = pred_df["id"].astype(str).str.strip()
gold_df["id"] = gold_df["id"].astype(str).str.strip()

# Merge
merged = pred_df.merge(gold_df, on="id", suffixes=("_pred", "_gold"))
print(f"\n✅ Merged: {len(merged)} / {len(gold_df)}")

if merged.empty:
    raise ValueError("❌ No matching IDs between prediction and gold files!")

y_true = merged["label_gold"]
y_pred = merged["label_pred"]

# 🎯 Basic Metrics
accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)
micro_f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

# 📐 Precision, Recall
precision = precision_score(y_true, y_pred, average="binary", zero_division=0)
recall = recall_score(y_true, y_pred, average="binary", zero_division=0)

# 🧾 Confusion Matrix & Report
conf_matrix = confusion_matrix(y_true, y_pred)
class_report = classification_report(y_true, y_pred, digits=4)

# 📊 Print Results
print("\n📊 Evaluation Metrics")
print(f"Accuracy     : {accuracy:.5f}")
print(f"Macro-F1     : {macro_f1:.5f}")
print(f"Micro-F1     : {micro_f1:.5f}")
print(f"Precision    : {precision:.5f}")
print(f"Recall       : {recall:.5f}")

print("\n📉 Confusion Matrix:")
print(conf_matrix)

print("\n📝 Classification Report:")
print(class_report)



✅ Merged: 34272 / 34272

📊 Evaluation Metrics
Accuracy     : 0.81772
Macro-F1     : 0.81403
Micro-F1     : 0.81772
Precision    : 0.77852
Recall       : 0.91256

📉 Confusion Matrix:
[[11599  4673]
 [ 1574 16426]]

📝 Classification Report:
              precision    recall  f1-score   support

           0     0.8805    0.7128    0.7878     16272
           1     0.7785    0.9126    0.8402     18000

    accuracy                         0.8177     34272
   macro avg     0.8295    0.8127    0.8140     34272
weighted avg     0.8269    0.8177    0.8154     34272



In [ ]:
!python /content/scorer.py --gold_file_path=/content/subtaskA_gold_labels.jsonl --pred_file_path=/content/Results.jsonl

INFO:root:✅ Prediction file format is valid.
✅ Merged rows: 34272 / 34272

📊 Evaluation Results:
Macro-F1:  0.81403
Micro-F1:  0.81772
Accuracy:  0.81772
